# Sales Pipeline Summary & Risk Flagging

**User Story**: US2 — Pipeline Summary & Risk Flagging
**Persona**: Sales Manager

This notebook demonstrates how to use the Salesforce AI Assistant to generate
a team pipeline summary with at-risk deals flagged. The assistant analyzes
pipeline data grouped by AE, identifies deals with risk indicators (stalled stage,
overdue close date, no recent activity), and provides actionable recommendations.

## Prerequisites
- `.env` file configured with Salesforce and Azure AI credentials
- `salesforce-crm` MCP server available
- Python packages installed (`pip install -r requirements.txt`)
- Risk thresholds configured in `config/risk_thresholds.yaml`

In [1]:
# Cell 2: Environment Check
import os
import sys

sys.path.insert(0, os.path.abspath(".."))

from dotenv import load_dotenv

load_dotenv("../.env")

required_vars = [
    "AZURE_AI_PROJECT_ENDPOINT",
    "AZURE_OPENAI_DEPLOYMENT",
    "SF_INSTANCE_URL",
    "SF_ACCESS_TOKEN",
]
missing = [v for v in required_vars if not os.environ.get(v)]
if missing:
    raise OSError(f"Missing environment variables: {', '.join(missing)}")

print("✅ Environment configured")
print(f"   Project: {os.environ['AZURE_AI_PROJECT_ENDPOINT'][:50]}...")
print(f"   SF Instance: {os.environ['SF_INSTANCE_URL']}")

✅ Environment configured
   Project: https://newfoundrydemoproject.services.ai.azure.co...
   SF Instance: https://orgfarm-181aeeadf6-dev-ed.develop.my.salesforce.com


In [ ]:
# Cell 3: Start MCP Server + Configure Connection
import subprocess
import time

from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import MCPTool
from azure.identity import DefaultAzureCredential

# Start the salesforce-crm MCP server locally with SSE transport
MCP_PORT = 8101
mcp_process = subprocess.Popen(
    [sys.executable, "-m", "mcp_servers.salesforce_crm.server"],
    env={**os.environ, "MCP_TRANSPORT": "sse", "FASTMCP_PORT": str(MCP_PORT)},
    cwd=os.path.abspath(".."),
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)
time.sleep(3)

if mcp_process.poll() is not None:
    stderr = mcp_process.stderr.read().decode() if mcp_process.stderr else ""
    raise RuntimeError(f"MCP server failed to start: {stderr}")

MCP_SERVER_URL = f"http://127.0.0.1:{MCP_PORT}/sse"
print(f"✅ MCP server started (PID: {mcp_process.pid}) at {MCP_SERVER_URL}")

# Create AI Project client
project = AIProjectClient(
    endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

# Create MCP tool pointing to the local SSE server
mcp_tool = MCPTool(server_label="salesforce-crm", server_url=MCP_SERVER_URL)
print(f"✅ MCP tool configured: {mcp_tool.server_label}")

✅ Authenticated with Azure AI Foundry


In [ ]:
# Cell 4: Create OpenAI client + load system prompt
from pathlib import Path as _Path

client = project.get_openai_client()
system_prompt = _Path("../agents/sales/system_prompt.md").read_text(encoding="utf-8")
MODEL = os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4o")

print(f"✅ OpenAI client ready (model: {MODEL})")
print(f"✅ System prompt loaded ({len(system_prompt)} chars)")

In [ ]:
# Cell 5: Pipeline Query with Risk Flags
from IPython.display import Markdown, display

response = client.responses.create(
    model=MODEL,
    instructions=system_prompt,
    input="Show me my team's pipeline and flag any at-risk deals. "
    "Group the summary by AE and highlight deals that are overdue, "
    "stalled, or have no recent activity.",
    tools=[mcp_tool],
)
print(f"✅ Response completed (id: {response.id})")

for item in response.output:
    if item.type == "message":
        for content in item.content:
            if content.type == "output_text":
                display(Markdown(content.text))

In [ ]:
# Cell 6: Risk-Flagged Deals Deep Dive
response2 = client.responses.create(
    model=MODEL,
    instructions=system_prompt,
    input="For the at-risk deals, what specific actions should each AE take this week?",
    tools=[mcp_tool],
    previous_response_id=response.id,
)

for item in response2.output:
    if item.type == "message":
        for content in item.content:
            if content.type == "output_text":
                display(Markdown(content.text))

In [ ]:
# Cell 7: AE-Specific Follow-up
AE_NAME = "Sarah"  # Change to a specific AE name

response3 = client.responses.create(
    model=MODEL,
    instructions=system_prompt,
    input=f"What should {AE_NAME} focus on this week?",
    tools=[mcp_tool],
    previous_response_id=response2.id,
)

for item in response3.output:
    if item.type == "message":
        for content in item.content:
            if content.type == "output_text":
                display(Markdown(content.text))

In [ ]:
# Cell 8: Cleanup — stop the local MCP server
mcp_process.terminate()
mcp_process.wait(timeout=5)
print(f"✅ MCP server stopped (PID: {mcp_process.pid})")
print("\n--- Session Complete ---")